[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github.com/schwartz-cnl/Computational-Neuroscience-Class/blob/main/Holographic%20Reduced%20Representations/HolographicReducedRepresentations_lab.ipynb)

In [ ]:
# Holographic Reduced Representations
#  References:
#  Plate, T. A. (1994).
#  Distributed Representations and Nested Compositional Structure.
#  Ph. D. thesis, Department of Computer Science, University of Toronto,
#  http://internet.cybermesa.com/~champagne/tplate/
#
#  Plate, T. A. (2003).
#  Holographic Reduced Representation: Distributed Representation for
#  Cognitive Structures.
#  Center for the Study of Language and Information (CSLI).
#  ISBN-10: 1575864290, April 2003

# ccorr_fft and cconv_fft originally by by Blerim Emruli, March 2012, in Matlab
# Main code originally by Odelia Schwartz in 2016 in Matlab
# Translated from Matlab to Python by ChatGPT-03-mini-high and verified by OS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def cconv_fft(x, y):
    """
    Perform circular convolution using FFT.

    Parameters:
        x, y (ndarray): Input vectors of equal length.

    Returns:
        z (ndarray): Normalized circular convolution of x and y.
    """
    if x.shape != y.shape:
        raise ValueError("vectors are not equally sized")
    z = np.fft.ifft(np.fft.fft(x) * np.fft.fft(y))
    z = z / np.linalg.norm(z)
    return z

def ccorr_fft(z, y):
    """
    Perform circular correlation using FFT.

    Parameters:
        z, y (ndarray): Input vectors of equal length.

    Returns:
        xTilde (ndarray): Normalized circular correlation of z and y.
    """
    if z.shape != y.shape:
        raise ValueError("vectors are not equally sized")
    xTilde = np.fft.ifft(np.fft.fft(z) * np.conj(np.fft.fft(y)))
    xTilde = xTilde / np.linalg.norm(xTilde)
    return xTilde


In [ ]:
# Main code
dVectors = 1000
np.random.seed(0)  # Optional: for reproducible results

# Generate random vectors with mean 0 and standard deviation sqrt(1/dVectors)
shape = np.random.normal(0, np.sqrt(1/dVectors), dVectors)
square = np.random.normal(0, np.sqrt(1/dVectors), dVectors)
circle = np.random.normal(0, np.sqrt(1/dVectors), dVectors)


In [ ]:
# Bind shape to circle in a vector called convshape
# Circular convolution is used for binding.
# We can think of this as holding a circle shape in memory (encoding)
# (the circle is an example shape and unrelated to circular convolution)
convshape = cconv_fft(shape, circle)

# Note that the circular convolution retains the same size
print("Shapes:", circle.shape, shape.shape, convshape.shape)


In [ ]:
# From bound vector convshape find what was bound to shape to obtain it
# This is the inverse operation that gives back a noisy circle.
# We will call this vector findshape. This is like decoding or
# retrieving from memory.

# Decode: retrieve what was bound to 'shape' (this gives a noisy circle)
findshape = ccorr_fft(convshape, shape)

# Let's compare the original circle to the decoded findshape
# Plot the first nn elements of the original circle and the decoded vector
nn = 50
plt.figure(1)
plt.plot(circle[:nn], label='circle')
plt.plot(findshape[:nn].real, 'r', label='decoded shape')
plt.legend()
plt.title("Circle vs. Decoded Shape")

In [ ]:
# Let's compare the square to the decoded findshape -- why does it look worse?
plt.figure(2)
plt.plot(square[:nn], label='square')
plt.plot(findshape[:nn].real, 'r', label='decoded shape')
plt.legend()
plt.title("Square vs. Decoded Shape")


In [ ]:
# Is the decoded findshape more similar to the square or to the circle?
# We can take the inner product: Which is higher?

# Inner products to check similarity
inner_product_square = np.dot(square, findshape.real)
inner_product_circle = np.dot(circle, findshape.real)
print("Inner product (square * decoded shape):", inner_product_square)
print("Inner product (circle * decoded shape):", inner_product_circle)

# An alternative way to decode is using the flipped shape vector
shapeinv = np.concatenate([shape[:1], np.flip(shape[1:])])
findshape2 = cconv_fft(convshape, shapeinv)
inner_product_square2 = np.dot(square, findshape2.real)
inner_product_circle2 = np.dot(circle, findshape2.real)
print("Inner product (square * findshape2):", inner_product_square2)
print("Inner product (circle * findshape2):", inner_product_circle2)


In [ ]:
# We can also bind a number together with the shape in "memory"
# For instance we want to hold "in memory" number one
# Bind a number and a vector 'one'
number = np.random.normal(0, np.sqrt(1/dVectors), dVectors)
one = np.random.normal(0, np.sqrt(1/dVectors), dVectors)
convnumber = cconv_fft(number, one)

# We want this together with the circle shape we computed earlier
# (as if holding one circle in memory)
convsum = convshape + convnumber

# From this sum, we again want to find what the shape was - square or circle
# Decode the shape from the sum
findshape_fromsum = ccorr_fft(convsum, shape)
plt.figure(3)
plt.plot(circle[:nn], label='circle')
plt.plot(findshape_fromsum[:nn].real, 'r', label='decoded shape from sum')
plt.legend()
plt.title("Circle vs. Decoded Shape from Sum")



In [ ]:
inner_product_square_sum = np.dot(square, findshape_fromsum.real)
inner_product_circle_sum = np.dot(circle, findshape_fromsum.real)
print("Inner product (square * decoded shape from sum):", inner_product_square_sum)
print("Inner product (circle * decoded shape from sum):", inner_product_circle_sum)


In [ ]:
# The two decoded shapes are similar, whether we just convolved circle with
# shape, or if we summed to together the circle and the one convolutions
plt.figure(4)
plt.plot(findshape[:nn].real, 'r', label='findshape')
plt.plot(findshape_fromsum[:nn].real, 'm', label='findshape from sum')
plt.legend()
plt.title("Comparison of Decoded Shapes")


In [ ]:
# We could also retrieve the number and compare it to 'one'
findnumber_fromsum = ccorr_fft(convsum, number)
findnumber_fromsum = ccorr_fft(convsum, number)
inner_product_one = np.dot(one, findnumber_fromsum.real)
print("Inner product (one * findnumber_fromsum):", inner_product_one)

plt.show()